In [2]:
from timeit import default_timer as timer
import math
import json
import inspect
import concurrent.futures as cf  # doesn't work with sklearn
import pandas as pd
import numpy as np
import copy as copy
import statistics as stt
import seaborn as sns
sns.set_theme()
from os import system, getcwd, startfile
from os.path import join
from time import time
from scipy.io import arff
from sklearn import tree, datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
%matplotlib inline
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, Adam

In [3]:
# generate data

class main():
    def __init__(self):
        self.X = None
        self.y = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        print('class initialized')

    def get_data(self, dataset='random', show_fig=False, test_size=0.5, random_state=2018):
        def generate_random_data(n, m):
            X = np.random.rand(n, m)
            y = np.random.randint(2, size=n)
            return X, y

        def two_blob_data(size=100):
            # X
            np.random.seed(2000)
            X1 = np.random.normal(4.5, 1.3, size)
            Y1 = np.random.normal(2.0, 0.8, size) + X1
            X2 = X1 + np.random.normal(0.01, 0.3, size)
            Y2 = Y1 + 8.0
            X = np.append(X1, X2)
            Y = np.append(Y1, Y2)
            X = np.append(X, Y)
            X = X.reshape(2, size*2).T
            # y
            y1 = [1 for i in range(size)]
            y2 = [0 for i in range(size)]
            y = np.append(y1, y2)
            return X, y

        def import_wdbc():
            path = join(getcwd().rstrip(
                'src'), 'data/wdbc.data').replace('\\', '/')
            data = pd.read_csv(path, header=None)
            wdbc_columns = ['id', 'malignant',
                            'nucleus_mean', 'nucleus_se', 'nucleus_worst',
                            'texture_mean', 'texture_se', 'texture_worst',
                            'perimeter_mean', 'perimeter_se', 'perimeter_worst',
                            'area_mean', 'area_se', 'area_worst',
                            'smoothness_mean', 'smoothness_se', 'smoothness_worst',
                            'compactness_mean', 'compactness_se', 'compactness_worst',
                            'concavity_mean', 'concavity_se', 'concavity_worst',
                            'concave_pts_mean', 'concave_pts_se', 'concave_pts_worst',
                            'symmetry_mean', 'symmetry_se', 'symmetry_worst',
                            'fractal_dim_mean', 'fractal_dim_se', 'fractal_dim_worst']
            data.columns = wdbc_columns
            data['malignant'] = data['malignant'].map(
                lambda x: 0 if x == 'B' else 1)
            X = data.drop(['id', 'malignant'], axis=1).values
            s = StandardScaler()
            X = s.fit_transform(X)
            y = data['malignant'].values
            return X, y

        def import_MNIST():
            # not working
            digits = datasets.load_digits()
            X = digits.data
            y = digits.target
            return X, y
        
        if dataset == 'random':
            self.X, self.y = generate_random_data(1000, 100)
        elif dataset == 'blob':
            self.X, self.y = two_blob_data(1000)
        elif dataset == 'wbcd':
            self.X, self.y = import_wdbc()
        elif dataset == 'MNIST':
            self.X, self.y = import_MNIST()
        else:
            print('dataset not found')
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=test_size, random_state=random_state)
        
        if show_fig:
            plt.figure(figsize=(10, 10))
            plt.scatter(self.X[:, 0], self.X[:, 1], c=self.y)
            plt.show()

    def NN(self, model=1, epochs=10, batch_size=32):
        def data_preprocessing():
            # one hot encoding
            y_train = np_utils.to_categorical(self.y_train)
            y_test = np_utils.to_categorical(self.y_test)
            # # normalization
            # self.X_train = self.X_train.astype('float32')
            # self.X_test = self.X_test.astype('float32')
            # self.X_train /= 255
            # self.X_test /= 255
            return y_train, y_test

        def build_model1():
            model = Sequential()
            N_Features = self.X_train.shape[1]
            model.add(Dense(N_Features, activation='relu'))
            model.add(Dense(2, activation='softmax'))
            model.compile(loss='categorical_crossentropy',
                            optimizer='adam', metrics=['accuracy'])
            return model

        def build_model2():
            # https://stackoverflow.com/questions/61742556/valueerror-shapes-none-1-and-none-2-are-incompatible
            # https: // github.com/fchollet/deep-learning-with -python-notebooks/issues/157
            model = Sequential()
            N_Features = self.X_train.shape[1]
            model.add(Dense(N_Features, activation='relu'))
            model.add(Dense(1, activation='sigmoid'))
            model.add(Flatten())
            model.compile(loss='binary_crossentropy',
                          optimizer='adam', metrics=['accuracy'])
            return model

        def build_model3():
            # https: // stackoverflow.com/questions/37213388/keras-accuracy-does-not -change
            # https://keras.io/api/optimizers/
            model = Sequential()
            N_Features = self.X_train.shape[1]
            model.add(Dense(N_Features, activation='relu'))
            model.add(Dense(N_Features*2, activation='relu'))
            model.add(Dense(1, activation='sigmoid'))
            model.add(Flatten())
            opt = SGD(lr=0.00001)
            model.compile(loss='binary_crossentropy',
                          optimizer='Ftrl', metrics=['accuracy'])
            return model

        def evaluate_result(y_pred):
            y_pred = np.array([int(x < y) for [x, y] in y_pred])
            acc = accuracy_score(self.y_test, y_pred)
            prc = precision_score(self.y_test, y_pred)
            rec = recall_score(self.y_test, y_pred)
            f1 = f1_score(self.y_test, y_pred)
            print('accuracy: ', acc)
            print('precision: ', prc)
            print('recall: ', rec)
            print('f1: ', f1)
            return [acc, prc, rec, f1]

        y_train, y_test = data_preprocessing()

        if model == 1:
            model = build_model1()
        elif model == 2:
            model = build_model2()
        elif model == 3:
            model = build_model3()
        
        model.fit(self.X_train, y_train, validation_data=(
            self.X_test, y_test), epochs=epochs, batch_size=batch_size)
        print(model.summary())
        # https://stackoverflow.com/questions/49527159/how-to-get-the-output-shape-of-a-layer-in-keras
        # https://www.activestate.com/resources/quick-reads/what-is-a-keras-model/
        # https://stackoverflow.com/questions/45799474/keras-model-evaluate-vs-model-predict-accuracy-difference-in-multi-class-nlp-ta




    

# Do the following homework
----------------

1. Modify your first_ANN to have only one output neuron and make the necessary changes on your first_ANN.
    - make sure your program is running properly.
- Use Wisconsin breast cancer data for your first_ANN
- Try MNIST digit data with your first_ANN.   MNIST digit data set is available in:
    - //www.kaggle.com/c/digit-recognizer/data   or
    - datasets.load_digits() can read dataset comes with sklearn
        - you need to import datasets package to be capable ot load_digites:
            - from sklearn import datasets

## Q1 Modify ANN to have only one output neuron

In [10]:
m = main()
m.get_data(dataset='blob', show_fig=False)
m.NN(model=2, epochs=10, batch_size=32)

class initialized
Epoch 1/10
32/32 [==============================] - 1s 6ms/step - loss: 1.0046 - accuracy: 0.5000 - val_loss: 0.9147 - val_accuracy: 0.5000
Epoch 2/10
32/32 [==============================] - 0s 2ms/step - loss: 0.8516 - accuracy: 0.5000 - val_loss: 0.7979 - val_accuracy: 0.5000
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 0.7647 - accuracy: 0.5000 - val_loss: 0.7374 - val_accuracy: 0.5000
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 0.7232 - accuracy: 0.5000 - val_loss: 0.7108 - val_accuracy: 0.5000
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 0.7062 - accuracy: 0.5000 - val_loss: 0.7019 - val_accuracy: 0.5000
Epoch 6/10
32/32 [==============================] - 0s 2ms/step - loss: 0.7004 - accuracy: 0.5000 - val_loss: 0.6985 - val_accuracy: 0.5000
Epoch 7/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6980 - accuracy: 0.5000 - val_loss: 0.6970 - val_accuracy: 0.5000
Ep

## Q2 Use Wisconsin breast cancer data for your ANN

In [9]:
m = main()
m.get_data(dataset='wbcd', show_fig=False)
m.NN(model=3, epochs=10, batch_size=1)


class initialized
Epoch 1/10


C:\Users\dachu\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


284/284 [==============================] - 1s 2ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/10
284/284 [==============================] - 0s 1ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/10
284/284 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/10
284/284 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/10
284/284 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/10
284/284 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/10
284/284 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 8/10
284/28

## Q3 Use MNIST data.

In [8]:
m = main()
m.get_data(dataset='MNIST', show_fig=False)
m.NN(model=2, epochs=10, batch_size=1)


class initialized
Epoch 1/10
898/898 [==============================] - 2s 1ms/step - loss: 0.4703 - accuracy: 0.8688 - val_loss: 0.3725 - val_accuracy: 0.8991
Epoch 2/10
898/898 [==============================] - 1s 1ms/step - loss: 0.3560 - accuracy: 0.8956 - val_loss: 0.3478 - val_accuracy: 0.8982
Epoch 3/10
898/898 [==============================] - 1s 1ms/step - loss: 0.3410 - accuracy: 0.9000 - val_loss: 0.3359 - val_accuracy: 0.9000
Epoch 4/10
898/898 [==============================] - 1s 1ms/step - loss: 0.3345 - accuracy: 0.9000 - val_loss: 0.3325 - val_accuracy: 0.9000
Epoch 5/10
898/898 [==============================] - 1s 1ms/step - loss: 0.3321 - accuracy: 0.9000 - val_loss: 0.3321 - val_accuracy: 0.8991
Epoch 6/10
898/898 [==============================] - 1s 1ms/step - loss: 0.3324 - accuracy: 0.9000 - val_loss: 0.3320 - val_accuracy: 0.9000
Epoch 7/10
898/898 [==============================] - 1s 1ms/step - loss: 0.3303 - accuracy: 0.9000 - val_loss: 0.3297 - val_accur